In [1]:
import os
import numpy as np
import pandas as pd
from ipywidgets import interact, fixed, widgets
from tqdm import tqdm
# visulisation function
from functions.visualisations import (
    histogram_trace, plot_y_timeseries, 
    plot_ycorr_scatter, boxplot_weights,
    plot_linear_data
)

from types import MethodType

np.random.seed(2023)

In [2]:
def forward_pass(self,X):
    l1_z=np.dot(X,self.l1_weights)+self.l1_biases
    self.l1_output=self.sigmoid(l1_z)
    l2_z=np.dot(self.l1_output,self.l2_weights)+self.l2_biases
    self.l2_output=self.sigmoid(l2_z)

    return self.l2_output


def backward_pass(self,X,Y):
    l2_delta=(Y-self.l2_output)*(self.l2_output*(1-self.l2_output))
    l2_weights_delta=np.outer(self.l1_output,l2_delta)

    l1_delta=np.dot(l2_delta,self.l2_weights.T)*(self.l1_output*(1-self.l1_output))
    l1_weights_delta=np.outer(X,l1_delta)

    self.l2_weights+=self.lrate*l2_weights_delta
    self.l2_biases+=self.lrate*l2_delta
    self.l1_weights+=self.lrate*l1_weights_delta
    self.l1_biases+=self.lrate*l1_delta


In [17]:
class NeuralNetwork:
    def __init__(self,layers_sizes,learning_rate=0.01):
        self.input_num=layers_sizes[0]
        self.hidden_num=layers_sizes[1]
        self.output_num=layers_sizes[2]

        self.n_params=(self.input_num*self.hidden_num)*(self.hidden_num*self.output_num)+\
            self.hidden_num+self.output_num

        self.lrate=learning_rate

        self.initialise_network()

        self.forward_pass=MethodType(forward_pass,self)
        self.backward_pass=MethodType(backward_pass,self)
    
    def initialise_network(self):

        self.l1_weights=np.random.normal(
            loc=0,scale=1/np.sqrt(self.input_num),
            size=(self.input_num,self.hidden_num)
        )
        self.l1_biases=np.random.normal(
            loc=0,scale=1/np.sqrt(self.hidden_num),
            size=((self.hidden_num,))
        )
        self.l1_output=np.zeros((1,self.hidden_num))

        self.l2_weights=np.random.normal(
            loc=0,scale=1/np.sqrt(self.hidden_num),
            size=(self.hidden_num,self.output_num)
        )
        self.l2_biases=np.random.normal(
            loc=0,scale=1/np.sqrt(self.hidden_num),
            size=(self.output_num,)
        )
        self.l2_output=np.zeros((1,self.output_num))

    def evaluate_proposal(self,x_data,theta):
        self.decode(theta)
        size=x_data.shape[0]
        fx=np.zeros(size)
        #prob=np.zeros((size,self.output_num))

        for i in range(0,size):
            fx_tmp=self.forward_pass(x_data[i,])
            fx[i]=fx_tmp
        
        return fx


    def langevin_gradient(self,x_data,y_data,theta,depth):
        self.decode(theta)
        size=x_data.shape[0]

        for _ in range(0,depth):
            for ii in range(0,size):
                self.forward_pass(x_data[ii,])
                self.backward_pass(x_data[ii,],y_data[ii])
        theta_updated=self.encode()
        return theta_updated

    def sigmoid(self, x):
        return 1 / (1 + np.exp(-x))
    
    def encode(self):
        w1=self.l1_weights.ravel()
        w2=self.l2_weights.ravel()
        theta=np.concatenate([w1,w2,self.l1_biases,self.l2_biases])
        return theta

    def decode(self,theta):
        w_layer1size=self.input_num*self.hidden_num
        w_layer2size=self.hidden_num*self.output_num
        w_layer1=theta[0:w_layer1size]
        self.l1_weights=np.reshape(w_layer1,(self.input_num,self.hidden_num))
        w_layer2=theta[w_layer1size:w_layer1size+w_layer2size]
        self.l2_weights=np.reshape(w_layer2,(self.hidden_num,self.output_num))

        self.l1_biases=theta[w_layer1size+w_layer2size:w_layer1size+w_layer2size+self.hidden_num]
        self.l2_biases=theta[w_layer1size+w_layer2size+self.hidden_num:w_layer1size + w_layer2size + self.hidden_num + self.output_num]



In [22]:
def likelihood_function(self,theta,tausq,test=False):
    if test:
        x_data=self.x_test
        y_data=self.y_test
    else:
        x_data=self.x_data
        y_data=self.y_data
    
    model_prediction=self.model.evaluate_proposal(x_data,theta)
    model_simulation=model_prediction+np.random.normal(0,tausq,size=model_prediction.shape)
    accuracy=self.rmse(model_prediction,y_data)
    log_likelihood=np.sum(-0.5*np.log(2*np.pi*tausq)-0.5*np.square(y_data-model_prediction)/tausq)
    return [log_likelihood, model_prediction, model_simulation, accuracy]

def prior(self, sigma_squared, nu_1, nu_2, theta, tausq): 

    n_params = self.theta_size # number of parameters in model
    part1 = -1 * (n_params / 2) * np.log(sigma_squared)
    part2 = 1 / (2 * sigma_squared) * (sum(np.square(theta)))
    log_prior = part1 - part2 - (1 + nu_1) * np.log(tausq) - (nu_2 / tausq)
    return log_prior


In [29]:
def sampler(self):
    pos_theta=np.ones((self.n_samples,self.theta_size))
    pos_tau=np.ones((self.n_samples,1))
    pos_eta=np.ones((self.n_samples,1))

    pred_y=np.zeros((self.n_samples,self.x_data.shape[0]))
    sim_y=np.zeros((self.n_samples,self.x_data.shape[0]))
    rmse_data=np.zeros(self.n_samples)

    test_pred_y=np.ones((self.n_samples,self.x_test.shape[0]))
    test_sim_y=np.ones((self.n_samples,self.x_test.shape[0]))
    test_rmse_data=np.zeros(self.n_samples)

    theta=np.random.randn(self.theta_size)

    pred_y[0,]=self.model.evaluate_proposal(self.x_data,theta)

    eta=np.log(np.var(pred_y[0,]-self.y_data))
    tau_proposal=np.exp(eta)

    sigma_squared=self.sigma_squared
    nu_1=self.nu_1
    nu_2=self.nu_2

    prior_val=self.prior(sigma_squared,nu_1,nu_2,theta,tau_proposal)
    [likelihood, pred_y[0,], sim_y[0,], rmse_data[0]] = self.likelihood_function(theta, tau_proposal)

    n_accept=0
    n_langevin=0

    for ii in tqdm(np.arange(1,self.n_samples)):
        theta_proposal=theta+np.random.normal(0,self.step_theta,self.theta_size)
        lx=np.random.uniform(0,1,1)
        if(self.use_langevin_gradients is True) and (lx<self.l_prob):
            theta_gd=self.model.langevin_gradient(self.x_data,self.y_data,theta.copy(),self.sgd_depth)
            theta_proposal=np.random.normal(theta_gd,self.step_theta,self.theta_size)
            theta_proposal_gd=self.model.langevin_gradient(self.x_data,self.y_data,theta_proposal.copy(),self.sgd_depth)

            wc_delta=(theta-theta_proposal_gd)
            wp_delta=(theta_proposal-theta_gd)

            sigma_sq=self.step_theta
            first=-0.5*np.sum(wc_delta*wc_delta)/sigma_sq
            second=-0.5*np.sum(wp_delta*wp_delta)/sigma_sq
            diff_prop=first-second
            n_langevin+=1
        else:
            diff_prop=0
            theta_proposal=np.random.normal(theta,self.step_theta)

    eta_proposal=eta+np.random.normal(0,self.step_eta,1)
    tau_proposal=np.exp(eta_proposal)

    prior_proposal=self.prior(sigma_squared,nu_1,nu_2,theta_proposal,tau_proposal)
    [likelihood_proposal, pred_y[ii,], sim_y[ii,], rmse_data[ii]] = self.likelihood_function(
        theta_proposal, tau_proposal
    )

    [_, test_pred_y[ii,], test_sim_y[ii,], test_rmse_data[ii]] = self.likelihood_function(
            theta_proposal, tau_proposal, test=True
        )
    diff_likelihood=likelihood_proposal-likelihood
    diff_prior=prior_proposal-prior_val
    mh_prob=min(1,np.exp(diff_likelihood+diff_prior+diff_prop))
    u=np.random.uniform(0,1)

    if u < mh_prob:
        n_accept+=1
        likelihood=likelihood_proposal
        prior_val=prior_proposal
        theta=theta_proposal
        eta=eta_proposal
        pos_theta[ii,]=theta_proposal
        pos_tau[ii,]=tau_proposal
        pos_eta[ii,]=eta_proposal
    else:
        pos_theta[ii,]=pos_theta[ii-1,]
        pos_tau[ii,]= pos_tau[ii-1,]
        pos_eta[ii,]= pos_eta[ii-1,]

    accept_ratio=(n_accept/self.n_samples)*100
    print('{:.3}% were acepted'.format(accept_ratio))
    
    self.pos_theta = pos_theta[self.n_burnin:, ]
    
    self.pos_tau = pos_tau[self.n_burnin:, ]
    
    self.pos_eta = pos_eta[self.n_burnin:, ]
    

    results_dict = {'w{}'.format(_): self.pos_theta[:, _].squeeze() for _ in range(self.theta_size-2)}
    results_dict['b0'] = self.pos_theta[:, self.theta_size-2].squeeze()
    results_dict['b1'] = self.pos_theta[:, self.theta_size-1].squeeze()    
    results_dict['tau'] = self.pos_tau.squeeze()
 
    # return the predictions
    pred_dict = {}
    pred_dict['train_pred'] = pred_y[self.n_burnin:,:]
    pred_dict['train_sim'] = sim_y[self.n_burnin:,:]
    pred_dict['test_pred'] = test_pred_y[self.n_burnin:,:]
    pred_dict['test_sim'] = test_sim_y[self.n_burnin:,:]
    
    results_df = pd.DataFrame.from_dict(
        results_dict
    )

    return results_df, pred_dict



In [7]:
class MCMC: 
    def __init__(self,model,n_samples,n_burnin,x_data,y_data,x_test,y_test):
        self.n_samples = n_samples # number of MCMC samples
        self.n_burnin = n_burnin # number of burn-in samples
        self.x_data = x_data # (N x num_features)
        self.y_data = y_data # (N x 1)
        self.x_test = x_test # (Nt x num_features)
        self.y_test = y_test # (Nt x 1)

        # MCMC sampler hyperparameters - defines how much variation you need in changes to theta, tau
        self.step_theta = 0.025;  
        self.step_eta = 0.2; # note eta is used as tau in the sampler to consider log scale.
        # Hyperpriors
        self.sigma_squared = 25
        self.nu_1 = 0
        self.nu_2 = 0
        self.model = model
        self.use_langevin_gradients = True
        self.sgd_depth = 1
        self.l_prob = 0.5 # likelihood prob
        self.theta_size = self.model.n_params # weights for each feature and a bias term

        # store output
        self.pos_theta = None
        self.pos_tau = None
        self.pos_eta = None
        self.rmse_data = None

        # functions defined above - this is poor practice, but done for readability 
        # and clarity
        self.likelihood_function = MethodType(likelihood_function, self)
        self.prior = MethodType(prior, self)
        self.sampler = MethodType(sampler, self)

    def rmse(self, predictions, targets):
        '''
        Additional error metric - root mean square error
        '''
        return np.sqrt(((predictions - targets) ** 2).mean())
        


In [8]:
name="Sunspot"
train_data=np.loadtxt("data/{}/train.txt".format(name))
test_data=np.loadtxt("data/{}/test.txt".format(name))
print('Training data shape: {}'.format(train_data.shape))
print('Test data shape: {}'.format(test_data.shape))

Training data shape: (298, 5)
Test data shape: (198, 5)


In [30]:
## MCMC Settings and Setup
n_samples       = 5000 # number of samples to draw from the posterior
burn_in         = int(n_samples* 0.5) # number of samples to discard before recording draws from the posterior
hidden          = 10
learning_rate   = 0.01

x_data = train_data[:,:-1]
y_data = train_data[:,-1]
x_test = test_data[:,:-1]
y_test = test_data[:,-1]

layer_sizes = [x_data.shape[1], hidden, 1]

nn_model = NeuralNetwork(layer_sizes,learning_rate)
mcmc = MCMC(nn_model,n_samples, burn_in, x_data, y_data, x_test, y_test)

result,pred=mcmc.sampler()


/var/folders/5k/43g5ft3d50d1_ftbvq8_x9rc0000gn/T/ipykernel_12116/446482416.py:47: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  fx[i]=fx_tmp
  0%|          | 0/4999 [00:00<?, ?it/s]


ValueError: shape mismatch: objects cannot be broadcast to a single shape.  Mismatch is between arg 0 with shape (411,) and arg 1 with shape (61,).